# DSC 180B CNN Notebook (Kaggle Dataset)

### Importing Needed Libraries

In [16]:
!pip uninstall opencv-python-headless -y 
!pip uninstall opencv-python -y
!pip uninstall opencv-contrib-python -y
!pip install opencv-python-headless
!pip install kaggle

Found existing installation: opencv-python-headless 4.11.0.86
Uninstalling opencv-python-headless-4.11.0.86:
  Successfully uninstalled opencv-python-headless-4.11.0.86
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (50.0 MB)


In [22]:
import json
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import torch
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
from pathlib import Path
import zipfile
from torchvision import models
import torch.nn as nn
from tqdm import tqdm
import torchvision.transforms as transforms
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from PIL import Image

In [24]:
def setup_kaggle(username: str, key: str):
    """
    Configures Kaggle API access and downloads the Chest X-ray Pneumonia dataset.
    :param username: Kaggle username
    :param key: Kaggle API key
    """
    # Define the kaggle credentials path
    kaggle_dir = Path.home() / ".kaggle"  # Use the user's home directory
    kaggle_dir.mkdir(parents=True, exist_ok=True)
    kaggle_json_path = kaggle_dir / "kaggle.json"
    
    # Write Kaggle credentials
    kaggle_credentials = {"username": username, "key": key}
    with open(kaggle_json_path, "w") as f:
        json.dump(kaggle_credentials, f)
    
    # Set permissions for security
    os.chmod(kaggle_json_path, 0o600)
    
    # Ensure kaggle CLI is in the path
    os.environ["KAGGLE_CONFIG_DIR"] = str(kaggle_dir)
        
    # Download the dataset
    dataset_path = Path("chest_xray_pneumonia")
    dataset_path.mkdir(exist_ok=True)

    os.system(f"kaggle datasets download -d paultimothymooney/chest-xray-pneumonia -p {dataset_path} --unzip")
    
    print("Dataset downloaded successfully!")
    
    # Define the unpack path
    unpack_path = Path("../../data/raw")
    unpack_path.mkdir(parents=True, exist_ok=True)
    
    # Unpack the zipped dataset into the specified path
    zip_file = dataset_path / "chest-xray-pneumonia.zip"
    
    if zip_file.exists():
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(unpack_path)
        print(f"Dataset unpacked successfully to {unpack_path}")
    else:
        print("Zipped dataset not found!")

# Get user input for Kaggle credentials
username = input("Enter your Kaggle Username: ")
key = input("Enter your generated Kaggle API key: ")

setup_kaggle(username, key)

Enter your Kaggle Username:  rohanduvur
Enter your generated Kaggle API key:  27b02a7a4196cad7e52cef5689776e91


Dataset downloaded successfully!


sh: 1: kaggle: Permission denied
